<a href="https://colab.research.google.com/github/SomdeepAcharyya/Recommender-Systems/blob/main/DCDCSR_A_Deep_Framework_for_Cross_Domain_and_Cross_System_Recommendations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Headers

In [ ]:
import pandas as pd
import scipy
!pip install implicit
import numpy as np
!pip install surprise
!pip install pandas_read_xml 
import pandas_read_xml as pdx

## Data Reading

In [ ]:
# Amazon books Dataset
path = r'/content/drive/MyDrive/Per_CD_RS/Amazon_Books.csv'
with open(path, encoding="utf-8", errors='ignore') as infile:
  amazon_books = pd.read_csv(infile)
  amazon_books = amazon_books[['user_id', 'item_id', 'rating']]
  amazon_books = amazon_books.rename(columns={"user_id":"userId", "item_id": "itemId"})
  amazon_books.columns
amazon_books.head()

,userId,itemId,rating
0,A1REUF3A1YCPHM,1713353,5
1,A1YRBRK2XM5D5,1713353,5
2,A1V8ZR5P78P4ZU,1713353,5
3,A2ZB06582NXCIV,1713353,5
4,ACPQVNRD3Z09X,1713353,5


In [ ]:
# Amazon Movies Dataset
path = r'/content/drive/MyDrive/Per_CD_RS/Amazon_Movies.csv'
with open(path, encoding="utf-8", errors='ignore') as infile:
  amazon_movies = pd.read_csv(infile)
  amazon_movies = amazon_movies[['user_id', 'item_id', 'rating']]
  amazon_movies = amazon_movies.rename(columns={"user_id":"userId", "item_id": "itemId"})
  amazon_movies.columns
amazon_movies.head()

,userId,itemId,rating
0,A2VHSG6TZHU1OB,1527665,5
1,A23EJWOW1TLENE,1527665,5
2,A1KM9FNEJ8Q171,1527665,5
3,A38LY2SSHVHRYB,1527665,4
4,AHTYUW2H1276L,1527665,5


In [ ]:
# Amazon Clothes Dataset
path = r'/content/drive/MyDrive/Per_CD_RS/Amazon_Clothing.csv'
with open(path, encoding="utf-8", errors='ignore') as infile:
  amazon_clothes = pd.read_csv(infile)
  amazon_clothes = amazon_clothes[['user_id', 'item_id', 'rating']]
  amazon_clothes = amazon_clothes.rename(columns={"user_id":"userId", "item_id": "itemId"})
  amazon_clothes.columns
amazon_clothes.head()

In [ ]:
# Amazon Video Games Dataset
path = r'/content/drive/MyDrive/Per_CD_RS/Amazon_Video_Games.csv'
with open(path, encoding="utf-8", errors='ignore') as infile:
  amazon_vg = pd.read_csv(infile)
  amazon_vg = amazon_vg[['user_id', 'item_id', 'rating']]
  amazon_vg = amazon_vg.rename(columns={"user_id":"userId", "item_id": "itemId"})
  amazon_vg.columns
amazon_vg.head()

In [ ]:
# MovieLens Dataset

path = r'/content/drive/MyDrive/Per_CD_RS/movielens_ratings.csv'
with open(path, encoding="utf-8", errors='ignore') as infile:
  movielens = pd.read_csv(infile)
movielens = movielens[['userId', 'movieId', 'rating']]
movielens = movielens.rename(columns={"movieId": "itemId"})
movielens.columns

Index(['userId', 'itemId', 'rating'], dtype='object')

In [ ]:
# BookCrossing Dataset

path = r'/content/drive/MyDrive/Per_CD_RS/BX_Book_Ratings.csv'
with open(path, encoding="utf-8", errors='ignore') as infile:
  bookcrossing = pd.read_csv(infile)
bookcrossing = bookcrossing[['User-ID', 'ISBN', 'Book-Rating']]
bookcrossing = bookcrossing.rename(columns={"User-ID":"userId", "ISBN":"itemId", "Book-Rating": "rating"})
bookcrossing.columns

Index(['userId', 'itemId', 'rating'], dtype='object')

In [ ]:
# Tripadvisor Dataset

path = r'/content/drive/MyDrive/Per_CD_RS/tripadvisor_reviews.csv'
with open(path, encoding="utf-8", errors='ignore') as infile:
  tripadvisor = pd.read_csv(infile)
tripadvisor = tripadvisor[['username', 'taObject', 'rating']]
tripadvisor = tripadvisor.rename(columns={"username":"userId", "taObject":"itemId"})
tripadvisor.columns

Index(['userId', 'itemId', 'rating'], dtype='object')

In [ ]:
# amazon review dataset magazines csv
path = r'/content/drive/MyDrive/Per_CD_RS/Amazon_Text_Magazine_Subscriptions.csv'
with open(path, encoding="utf-8", errors='ignore') as infile:
  az = pd.read_csv(infile, nrows=12000)
az = az.rename(columns={"reviewerID":"userId", "asin":"itemId", "overall":"rating"})

In [ ]:
source_user_ratings = tripadvisor
target_user_ratings = az

In [ ]:
print(source_user_ratings.shape)
print(target_user_ratings.shape)

(32580, 3)
(12000, 13)


In [ ]:
# creation of target rating matx for matx factorization(MF)
rs = source_user_ratings.pivot_table(index='userId',columns='itemId',values='rating', fill_value=0)
rs.shape
# creation of target rating matx for matx factorization(MF)
rt = target_user_ratings.pivot_table(index='userId',columns='itemId',values='rating', fill_value=0)
rt.shape

(11295, 66)

## SVD matrix Factorization 

In [ ]:
# matrix factorization(MF) via singular value decomposition(SVD) ad bm25_weight

import scipy.sparse.linalg
from implicit.nearest_neighbours import bm25_weight
vt, x1, ut = scipy.sparse.linalg.svds(bm25_weight(rt), 50)
vs, x2, us = scipy.sparse.linalg.svds(bm25_weight(rs), 50)

/usr/local/lib/python3.7/dist-packages/implicit/gpu/__init__.py:14: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: no CUDA-capable device is detected (/project/./implicit/gpu/utils.h:71)'
  f"CUDA extension is built, but disabling GPU support because of '{e}'",


In [ ]:
ut.shape

(50, 66)

## ProbabilisticMatrixFactorization

In [ ]:
raw_t=target_user_ratings[['userId','itemId','rating']] 
raw_s=source_user_ratings[['userId','itemId','rating']]

In [ ]:
# matrix factorization(MF) via Probabilistic Matrix Factorization model
import surprise

reader = surprise.Reader(rating_scale=(1,5)) 
#into surprise:
data_t = surprise.Dataset.load_from_df(raw_t,reader)
data_s = surprise.Dataset.load_from_df(raw_s,reader)

class ProbabilisticMatrixFactorization(surprise.AlgoBase):
# Randomly initializes two Matrices, Stochastic Gradient Descent to be able to optimize the best factorization for ratings.
    def __init__(self,learning_rate,num_epochs,num_factors):
       # super(surprise.AlgoBase)
        self.alpha = learning_rate #learning rate for Stochastic Gradient Descent
        self.num_epochs = num_epochs
        self.num_factors = num_factors
    def fit(self,train):
        #randomly initialize user/item factors from a Gaussian
        P = np.random.normal(0,.1,(train.n_users,self.num_factors))
        Q = np.random.normal(0,.1,(train.n_items,self.num_factors))
        #print('fit')

        for epoch in range(self.num_epochs):
            for u,i,r_ui in train.all_ratings():
                residual = r_ui - np.dot(P[u],Q[i])
                temp = P[u,:] # we want to update them at the same time, so we make a temporary variable. 
                P[u,:] +=  self.alpha * residual * Q[i]
                Q[i,:] +=  self.alpha * residual * temp 

                
        self.P = P
        self.Q = Q

        self.trainset = train
    
    
    def estimate(self,u,i):
        #returns estimated rating for user u and item i. Prerequisite: Algorithm must be fit to training set.
        #check to see if u and i are in the train set:
        #print('gahh')

        if self.trainset.knows_user(u) and self.trainset.knows_item(i):
            #print(u,i, '\n','yep:', self.P[u],self.Q[i])
            #return scalar product of P[u] and Q[i]
            nanCheck = np.dot(self.P[u],self.Q[i])
            
            if np.isnan(nanCheck):
                return self.trainset.global_mean
            else:
                return np.dot(self.P[u,:],self.Q[i,:])
        else:# if its not known we'll return the general average. 
           # print('global mean')
            return self.trainset.global_mean

In [ ]:
Alg_t = ProbabilisticMatrixFactorization(learning_rate=0.001,num_epochs=10,num_factors=10)
data_t1 = data_t.build_full_trainset()
Alg_t.fit(data_t1)
print(raw_t.userId.iloc[4],raw_t.itemId.iloc[4], raw_t.rating.iloc[4])
Alg_t.estimate(raw_t.userId.iloc[4], raw_t.itemId.iloc[4])

A25MDGOMZ2GALN B00005N7P0 5


3.949666666666667

In [ ]:
Alg_s = ProbabilisticMatrixFactorization(learning_rate=0.001,num_epochs=10,num_factors=10)
data_s1 = data_s.build_full_trainset()
Alg_s.fit(data_s1)
print(raw_s.userId.iloc[4],raw_s.itemId.iloc[4], raw_s.rating.iloc[4])
Alg_s.estimate(raw_s.userId.iloc[4], raw_s.itemId.iloc[4])

007solotraveler Hunter Steakhouse 5


4.039318600368324

In [ ]:
Alg_s.fit(data_s1)

In [ ]:
ut = Alg_t.P
us = Alg_s.P
vt = Alg_t.Q
vs = Alg_s.Q

In [ ]:
ut[1]

array([ 0.11792063,  0.00571279, -0.0473595 ,  0.00876846,  0.0645016 ,
        0.03143918, -0.0882703 ,  0.19890237, -0.12415067,  0.00058347])

## Bayesian Personalised Ranking Model Matrix Factorization

In [ ]:
import sys
import numpy as np
import pandas as pd
from math import ceil
from tqdm import trange
from subprocess import call
from itertools import islice
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import normalize
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix, dok_matrix

In [ ]:
def create_matrix(data, users_col, items_col, ratings_col, threshold = None):
      if threshold is not None:
        data = data[data[ratings_col] >= threshold]
        data[ratings_col] = 1
    
      for col in (items_col, users_col, ratings_col):
          data[col] = data[col].astype('category')

      ratings = csr_matrix((data[ratings_col],
                            (data[users_col].cat.codes, data[items_col].cat.codes)))
      ratings.eliminate_zeros()
      return ratings, data

items_col = 'itemId'
users_col = 'userId'
ratings_col = 'rating'
threshold = 3
X_t, df_t = create_matrix(target_user_ratings, users_col, items_col, ratings_col, threshold)
X_t
X_s, df_s = create_matrix(source_user_ratings, users_col, items_col, ratings_col, threshold)
X_s

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

<1098x24294 sparse matrix of type '<class 'numpy.longlong'>'
	with 29207 stored elements in Compressed Sparse Row format>

In [ ]:
def create_train_test(ratings, test_size = 0.2, seed = 1234):
    assert test_size < 1.0 and test_size > 0.0

    # Dictionary Of Keys based sparse matrix is more efficient
    # for constructing sparse matrices incrementally compared with csr_matrix
    train = ratings.copy().todok()
    test = dok_matrix(train.shape)
    
    # for all the users assign randomly chosen interactions
    # to the test and assign those interactions to zero in the training;
    # when computing the interactions to go into the test set, 
    # remember to round up the numbers (e.g. a user has 4 ratings, if the
    # test_size is 0.2, then 0.8 ratings will go to test, thus we need to
    # round up to ensure the test set gets at least 1 rating)
    rstate = np.random.RandomState(seed)
    for u in range(ratings.shape[0]):
        split_index = ratings[u].indices
        n_splits = ceil(test_size * split_index.shape[0])
        test_index = rstate.choice(split_index, size = n_splits, replace = False)
        test[u, test_index] = ratings[u, test_index]
        train[u, test_index] = 0
    
    train, test = train.tocsr(), test.tocsr()
    return train, test

X_t_train, X_t_test = create_train_test(X_t, test_size = 0.2, seed = 1234)
X_t_train
X_s_train, X_s_test = create_train_test(X_s, test_size = 0.2, seed = 1234)
X_s_train

<1098x24294 sparse matrix of type '<class 'numpy.longlong'>'
	with 22916 stored elements in Compressed Sparse Row format>

In [ ]:
class BPR:
    def __init__(self, learning_rate = 0.01, n_factors = 15, n_iters = 10, 
                 batch_size = 1000, reg = 0.01, seed = 1234, verbose = True):
        self.reg = reg
        self.seed = seed
        self.verbose = verbose
        self.n_iters = n_iters
        self.n_factors = n_factors
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        
        # to avoid re-computation at predict
        self._prediction = None
        
    def fit(self, ratings):
        
        indptr = ratings.indptr
        indices = ratings.indices
        n_users, n_items = ratings.shape
        
        # ensure batch size makes sense, since the algorithm involves
        # for each step randomly sample a user, thus the batch size
        # should be smaller than the total number of users or else
        # we would be sampling the user with replacement
        batch_size = self.batch_size
        if n_users < batch_size:
            batch_size = n_users
            sys.stderr.write('WARNING: Batch size is greater than number of users,'
                             'switching to a batch size of {}\n'.format(n_users))

        batch_iters = n_users // batch_size
        
        # initialize random weights
        rstate = np.random.RandomState(self.seed)
        self.user_factors = rstate.normal(size = (n_users, self.n_factors))
        self.item_factors = rstate.normal(size = (n_items, self.n_factors))
        
        # progress bar for training iteration if verbose is turned on
        loop = range(self.n_iters)
        if self.verbose:
            loop = trange(self.n_iters, desc = self.__class__.__name__)
        
        for _ in loop:
            for _ in range(batch_iters):
                sampled = self._sample(n_users, n_items, indices, indptr)
                sampled_users, sampled_pos_items, sampled_neg_items = sampled
                self._update(sampled_users, sampled_pos_items, sampled_neg_items)

        return self
    
    def _sample(self, n_users, n_items, indices, indptr):
        """sample batches of random triplets u, i, j"""
        sampled_pos_items = np.zeros(self.batch_size, dtype = np.int)
        sampled_neg_items = np.zeros(self.batch_size, dtype = np.int)
        sampled_users = np.random.choice(
            n_users, size = self.batch_size, replace = False)

        for idx, user in enumerate(sampled_users):
            pos_items = indices[indptr[user]:indptr[user + 1]]
            pos_item = np.random.choice(pos_items)
            neg_item = np.random.choice(n_items)
            while neg_item in pos_items:
                neg_item = np.random.choice(n_items)

            sampled_pos_items[idx] = pos_item
            sampled_neg_items[idx] = neg_item

        return sampled_users, sampled_pos_items, sampled_neg_items
                
    def _update(self, u, i, j):
        """
        update according to the bootstrapped user u, 
        positive item i and negative item j
        """
        user_u = self.user_factors[u]
        item_i = self.item_factors[i]
        item_j = self.item_factors[j]

        r_uij = np.sum(user_u * (item_i - item_j), axis = 1)
        sigmoid = np.exp(-r_uij) / (1.0 + np.exp(-r_uij))
        
        # repeat the 1 dimension sigmoid n_factors times so
        # the dimension will match when doing the update
        sigmoid_tiled = np.tile(sigmoid, (self.n_factors, 1)).T

        # update using gradient descent
        grad_u = sigmoid_tiled * (item_j - item_i) + self.reg * user_u
        grad_i = sigmoid_tiled * -user_u + self.reg * item_i
        grad_j = sigmoid_tiled * user_u + self.reg * item_j
        self.user_factors[u] -= self.learning_rate * grad_u
        self.item_factors[i] -= self.learning_rate * grad_i
        self.item_factors[j] -= self.learning_rate * grad_j
        return self

    def predict(self):
        """
        Obtain the predicted ratings for every users and items
        by doing a dot product of the learnt user and item vectors.
        The result will be cached to avoid re-computing it every time
        we call predict, thus there will only be an overhead the first
        time we call it. Note, ideally you probably don't need to compute
        this as it returns a dense matrix and may take up huge amounts of
        memory for large datasets
        """
        if self._prediction is None:
            self._prediction = self.user_factors.dot(self.item_factors.T)

        return self._prediction

    def _predict_user(self, user):
        """
        returns the predicted ratings for the specified user,
        this is mainly used in computing evaluation metric
        """
        user_pred = self.user_factors[user].dot(self.item_factors.T)
        return user_pred

    def recommend(self, ratings, N = 5):
       
        n_users = ratings.shape[0]
        recommendation = np.zeros((n_users, N), dtype = np.uint32)
        for user in range(n_users):
            top_n = self._recommend_user(ratings, user, N)
            recommendation[user] = top_n

        return recommendation

    def _recommend_user(self, ratings, user, N):
        """the top-N ranked items for a given user"""
        scores = self._predict_user(user)

        # compute the top N items, removing the items that the user already liked
        # from the result and ensure that we don't get out of bounds error when 
        # we ask for more recommendations than that are available
        liked = set(ratings[user].indices)
        count = N + len(liked)
        if count < scores.shape[0]:

            ids = np.argpartition(scores, -count)[-count:]
            best_ids = np.argsort(scores[ids])[::-1]
            best = ids[best_ids]
        else:
            best = np.argsort(scores)[::-1]

        top_n = list(islice((rec for rec in best if rec not in liked), N))
        return top_n
    
    def get_similar_items(self, N = 5, item_ids = None):
        normed_factors = normalize(self.item_factors)
        knn = NearestNeighbors(n_neighbors = N + 1, metric = 'euclidean')
        knn.fit(normed_factors)

        # returns a distance, index tuple,
        # we don't actually need the distance
        if item_ids is not None:
            normed_factors = normed_factors[item_ids]

        _, items = knn.kneighbors(normed_factors)
        similar_items = items[:, 1:].astype(np.uint32)
        return similar_items

In [ ]:
bpr_params = {'reg': 0.01,
              'learning_rate': 0.1,
              'n_iters': 160,
              'n_factors': 15,
              'batch_size': 100}

bpr_t = BPR(**bpr_params)
bpr_s = BPR(**bpr_params)
bpr_t.fit(X_t_train)
bpr_s.fit(X_s_train)

BPR:   0%|          | 0/160 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:53: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release no

ValueError: ignored

In [ ]:
bpr.user_factors

In [ ]:
bpr.item_factors

In [ ]:
from sklearn.impute import SimpleImputer
data_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
# fit the imputer - suppose missing data is in the 0th column
data_imputer = data_imputer.fit(X_t_train)
# transform the data
data = data_imputer.transform(X_t_train)

In [ ]:
type(X_t_train)

In [ ]:
!pip install sklearn

# Dnn mapping

In [ ]:
ei_t = pd.DataFrame()
ei_s = pd.DataFrame()

In [ ]:
t = target_user_ratings['userId'].value_counts()
ei_t = t.to_frame()
ud = []
for a in range(len(ei_t)):
  rows = []
  for b in range(10):
    rows.append(0)
  ud.append(rows)
ei_t['latent_factor'] = ud
for i in range(len(ei_t)):
  ei_t['latent_factor'][i] = ut[i]
x = target_user_ratings.sort_values(by='userId')
x = np.array(x['userId'].drop_duplicates())
ei_t = ei_t.sort_index()
ei_t = ei_t.rename(columns={"userId": "ni"})
ei_t['userId'] = x
ei_t

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,ni,latent_factor,userId
A0034986DWR7WEDQN0GV,1,"[-0.12765465952427302, -0.17684285510750528, 0...",A0034986DWR7WEDQN0GV
A0160612BLIWRHROHLLE,1,"[0.07045441840735922, -0.18557763786227685, 0....",A0160612BLIWRHROHLLE
A032903147O439YA3R0D,1,"[-0.011696874023991076, -0.014122395600793545,...",A032903147O439YA3R0D
A100CQXJ6D44T9,1,"[0.07993089395999163, -0.0542324205675629, 0.1...",A100CQXJ6D44T9
A100MPD67FD3ID,1,"[-0.005536118080176104, 0.13938347357185474, -...",A100MPD67FD3ID
...,...,...,...
AZWLJC4YOJDYW,1,"[-0.012263824034034199, -0.17065938535497122, ...",AZWLJC4YOJDYW
AZWQOKJLSRFTB,1,"[-0.0121745599533533, -0.09485913442567812, -0...",AZWQOKJLSRFTB
AZY3B95680TUC,1,"[0.018329393908584162, 0.22391680820337428, -0...",AZY3B95680TUC
AZYYFN06QYPC9,1,"[-0.04118857697605252, 0.033883898676644986, -...",AZYYFN06QYPC9


In [ ]:
s = source_user_ratings['userId'].value_counts()
ei_s = s.to_frame()
ud = []
for a in range(len(ei_s)):
  rows = []
  for b in range(10):
    rows.append(0)
  ud.append(rows)
ei_s['latent_factor'] = ud
for i in range(len(ei_s)):
  ei_s['latent_factor'][i] = us[i]
x = source_user_ratings.sort_values(by='userId')
x = np.array(x['userId'].drop_duplicates())
ei_s = ei_s.sort_index()
ei_s = ei_s.rename(columns={"userId": "ni"})
ei_s['userId'] = x
ei_s

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,ni,latent_factor,userId
A0334855HN6E38CXWXZR,1,"[0.06338707424427943, -0.042142567095948165, 0...",A0334855HN6E38CXWXZR
A0520969JGW2CZRMEZMM,1,"[0.11541894481023358, 0.03421455786402636, 0.1...",A0520969JGW2CZRMEZMM
A0790722OCX87RKL2J3T,1,"[-0.08781880467675265, -0.04883149553448582, -...",A0790722OCX87RKL2J3T
A100ATCZ8R81KG,1,"[-0.16361300197173395, 0.13930260363589853, -0...",A100ATCZ8R81KG
A101M64MW6EH3O,1,"[-0.06286932110741884, 0.00992801506089835, -0...",A101M64MW6EH3O
...,...,...,...
AZVK41090G5LI,1,"[0.013165219112807856, -0.11239192953493417, -...",AZVK41090G5LI
AZVVRNCE2F0ZY,1,"[-0.010629509193683252, 0.09750171714690012, 0...",AZVVRNCE2F0ZY
AZWV6VDHSTION,1,"[-0.0051956922152628, 0.011702528786929171, 0....",AZWV6VDHSTION
AZY870M82TGEI,1,"[-0.04250248205542601, 0.13367436508139946, -0...",AZY870M82TGEI


### Common Users

In [ ]:
cu = pd.DataFrame()
common_users = np.intersect1d(np.array(ei_s['userId']), np.array(ei_t['userId']))

In [ ]:
cu_s = ei_s[ei_s['userId'].isin(common_users)]

In [ ]:
cu_t = ei_t[ei_t['userId'].isin(common_users)]

In [ ]:
cu = pd.DataFrame()
cu['userId'] = cu_s['userId']
cu['ni_s'] = cu_s['ni']
cu['ni_t'] = cu_t['ni']
cu['UI_s'] = cu_s['latent_factor']
cu['UI_t'] = cu_t['latent_factor']
cu

,userId,ni_s,ni_t,UI_s,UI_t
A1725TI23A8AZ5,A1725TI23A8AZ5,1,1,"[-0.02605857811307092, 0.014199546389354131, -...","[-0.06310286361546404, -0.10257794931370896, 0..."
A17FLA8HQOFVIG,A17FLA8HQOFVIG,1,2,"[-0.022268642981203304, -0.0511238684610821, -...","[-0.05730133137103992, 0.07942591345023191, 0...."
A18DX1AT72SY95,A18DX1AT72SY95,1,1,"[-5.930269002463878e-06, 0.04757114650796635, ...","[-0.0581236924091468, -0.17221666290376353, 0...."
A18OUQEK7IZ2F2,A18OUQEK7IZ2F2,2,1,"[0.0632379746764821, -0.029914292483542763, 0....","[-0.011592110326265097, 0.10531469422414447, -..."
A197527ILE0LL8,A197527ILE0LL8,2,1,"[-0.08084390981052827, 0.04719132296820089, -0...","[-0.05320966442779071, 0.04131776714985763, -0..."
A1BT9J2I6DC246,A1BT9J2I6DC246,1,1,"[0.007371742634319282, -0.047634698555889206, ...","[-0.11449400872882938, 0.07518294948195972, -0..."
A1G9FX1KV45N41,A1G9FX1KV45N41,2,1,"[0.0747607901416618, -0.027134423572731547, -0...","[0.026464316012560357, 0.05953429626594828, 0...."
A1IT24LJIUSU46,A1IT24LJIUSU46,1,1,"[0.07902382096682845, 0.01929603262048468, -0....","[0.01606174841810614, 0.043418203465232626, 0...."
A1MUYL6VOFV40C,A1MUYL6VOFV40C,1,1,"[-0.05685106965029953, -0.036133899810993596, ...","[-0.040788651419980865, 0.005507241137412139, ..."
A1RPTVW5VEOSI,A1RPTVW5VEOSI,1,1,"[-0.11756091622780496, 0.027387818941186512, -...","[0.10377228219107865, -0.04682026739957298, 0...."


In [ ]:
cu['ai_s'] = 0.00
cu['ai_t'] = 0.00
for i, rows in cu.iterrows():
  x = rows['ni_t']/(rows['ni_s']+rows['ni_t'])
  y = 1-x
  cu.at[i,'ai_s'] = x
  cu.at[i,'ai_t'] = y

In [ ]:
ud = []
for a in range(len(cu)):
  rows = []
  for b in range(10):
    rows.append(0)
  ud.append(rows)
cu['CU_b'] = ud

for i in range(len(cu)):
  for j in range(10):
    cu['CU_b'][i][j] = (1-cu['ai_s'][i])*cu['UI_s'][i][j]+(1-cu['ai_t'][i])*cu['UI_t'][i][j]

cu['CU_b']

A1725TI23A8AZ5    [-0.04458072086426748, -0.04418920146217742, -...
A17FLA8HQOFVIG    [-0.04562376857442771, 0.03590931947979391, -0...
A18DX1AT72SY95    [-0.029064811339074632, -0.06232275819789859, ...
A18OUQEK7IZ2F2    [0.0382946130088997, 0.015162036419019635, 0.0...
A197527ILE0LL8    [-0.07163249468294909, 0.045233471028753136, -...
A1BT9J2I6DC246    [-0.05356113304725505, 0.013774125463035258, -...
A1G9FX1KV45N41    [0.058661965431961324, 0.0017551497068283885, ...
A1IT24LJIUSU46    [0.047542784692467296, 0.03135711804285865, 0....
A1MUYL6VOFV40C    [-0.048819860535140196, -0.015313329336790728,...
A1RPTVW5VEOSI     [-0.006894317018363155, -0.009716224229193233,...
A1SHMC6IXYLFAP    [0.004318845958482559, 0.10448402171240204, -0...
A1TAIDCNXTZV1F    [-0.08553017600786332, -0.07681340890137708, 0...
A1TKYAYESYHIA2    [0.10312403539295384, 0.04281293531989589, 0.1...
A1TMAVN4CEM8U8    [0.021588590379081368, -0.03594069627642432, -...
A1TPW86OHXTXFC    [-0.08565005274151302, 0.02969

## Different Users

In [ ]:
ei_s.columns

Index(['ni', 'latent_factor', 'userId'], dtype='object')

In [ ]:
du_s = ei_s[~ei_s['userId'].isin(common_users)]
du_t = ei_t[~ei_t['userId'].isin(common_users)]
frames = [du_s, du_t]
du = pd.concat(frames)
du.shape

(19382, 3)

In [ ]:
du_t

,ni,latent_factor,userId
A0034986DWR7WEDQN0GV,1,"[-0.12765465952427302, -0.17684285510750528, 0...",A0034986DWR7WEDQN0GV
A0160612BLIWRHROHLLE,1,"[0.07045441840735922, -0.18557763786227685, 0....",A0160612BLIWRHROHLLE
A032903147O439YA3R0D,1,"[-0.011696874023991076, -0.014122395600793545,...",A032903147O439YA3R0D
A100CQXJ6D44T9,1,"[0.07993089395999163, -0.0542324205675629, 0.1...",A100CQXJ6D44T9
A100MPD67FD3ID,1,"[-0.005536118080176104, 0.13938347357185474, -...",A100MPD67FD3ID
...,...,...,...
AZWLJC4YOJDYW,1,"[-0.012263824034034199, -0.17065938535497122, ...",AZWLJC4YOJDYW
AZWQOKJLSRFTB,1,"[-0.0121745599533533, -0.09485913442567812, -0...",AZWQOKJLSRFTB
AZY3B95680TUC,1,"[0.018329393908584162, 0.22391680820337428, -0...",AZY3B95680TUC
AZYYFN06QYPC9,1,"[-0.04118857697605252, 0.033883898676644986, -...",AZYYFN06QYPC9


In [ ]:
from scipy.spatial.distance import cdist #no use
import pandas as pd
k = 5000
XA = du['latent_factor']
XB = ei_s['latent_factor']
XA = np.array(XA.values.tolist())
print(XA.shape)
XB = np.array(XB.values.tolist())
print(XB.shape)
t = []
t = cdist(XA[k:k+9999], XB, 'euclidean')
t

In [ ]:
du['sni'] = 0.00  # no use
s = k
for i in range(len(t)):
  n = 5
  x = []
  sni = []
  idx = np.argpartition(t[i], k)
  x = idx[0:n]
  for j in range(len(x)):
    sni.append(ei_s['ni'][j])
  du['sni'][s] = sum(sni)/len(sni)
  s = s+1
  du['sni']

In [ ]:
du['ni_t'] = 0
du['ni_s'] = 0
for i in range(len(du)):
  x = du['userId'][i]
  for j in range(len(du_t)):
    if x == du_t['userId'][j]:
      du['ni_t'][i] = du_t['ni'][j]
  for j in range(len(du_s)):
    if x == du_s['userId'][j]:
      du['ni_s'][i] = du_s['ni'][j]
du

In [ ]:
ud = []
for a in range(len(du)):
  rows = []
  for b in range(10):
    rows.append(0)
  ud.append(rows)
du['UI_s'] = ud
du['UI_t'] = ud

for i in range(len(du)):
  x = du['userId'][i]
  row = []
  for j in range(len(du_t)):
    if x == du_t['userId'][j]:
      for k in range(10):
        row.append(du_t['latent_factor'][j][k])
      du['UI_t'][i] = row
  for j in range(len(du_s)):
    if x == du_s['userId'][j]:
      for k in range(10):
        row.append(du_s['latent_factor'][j][k])
      du['UI_s'][i] = row
du

In [ ]:
X = du[du['ni_t']==1]
du_t['ni_t'] = X['ni_t']
du_t['UI_t'] = X['UI_t']
du_t

In [ ]:
# calculate sni su beta du for different users 
from scipy.spatial import distance
import numpy
import pandas as pd
k = 5
count = 0
ud = []
for a in range(len(du_t)):
  rows = []
  for b in range(10):
    rows.append(0)
  ud.append(rows)

du_t['sni'] = 0.00
du_t['su'] = ud
du_t['bi_t'] = 0.00
du_t['du_b'] = ud

for i, r1 in du_t.iterrows():
  x = (pd.DataFrame())
  x['dist'] = 0.00
  x['sni'] = 0
  c = []
  d = []
  e = []
  count = count + 1
  for j, r2 in ei_s.iterrows():
    #x.at(j, 'dist') = distance.euclidean(r1['latent_factor'], r2['latent_factor'])
    #x.at(j, 'sni') = r2['ni']
    c.append((distance.euclidean(r1['latent_factor'], r2['latent_factor'])))
    d.append(r2['ni'])
    e.append(r2['latent_factor'])
  x['dist'] = c
  x['sni'] = d
  x['similar_users_latent_factor'] = e
  x.sort_values(by='dist')
  y = x[0:k]
  sim = y
  r = []
  for m in range(len(y)):
    r.append(numpy.multiply(1-y.dist[m], y.similar_users_latent_factor[m]))
  sim_u = numpy.sum(r, axis=0)
  du_t.at[i, 'su'] = numpy.divide(sim_u, (5-y['dist'].sum()))
  du_t.at[i,'sni'] = y['sni'].mean()
  du_t.at[i, 'bi_t'] = du_t['sni'][i]/(du_t['sni'][i]+du_t['ni'][i])
for i in range(len(du_t)):
  for j in range(10):
    du_t['du_b'][i][j] = (1-du_t['bi_t'][i])*du_t['latent_factor'][i][j]+(du_t['bi_t'][i])*du_t['su'][i][j]
  #r1['sni'] = y['sni'].mean()

In [ ]:
for i in range(len(du_t)):
  for j in range(10):
    du_t['du_b'][i][j] = (1-du_t['bi_t'][i])*du_t['latent_factor'][i][j]+(du_t['bi_t'][i])*du_t['su'][i][j]
  #r1['sni'] = y['sni'].mean()

du_t

,ni,latent_factor,userId,sni,su,bi_t,du_b
A0034986DWR7WEDQN0GV,1.0,"[-0.12765465952427302, -0.17684285510750528, 0...",A0034986DWR7WEDQN0GV,1.0,"[-0.017927479616481237, 0.01913447884337709, 0...",0.5,"[-0.07279106957037712, -0.07885418813206409, 0..."
A0160612BLIWRHROHLLE,1.0,"[0.07045441840735922, -0.18557763786227685, 0....",A0160612BLIWRHROHLLE,1.0,"[-0.01615093564490263, 0.018047639402592436, 0...",0.5,"[0.027151741381228296, -0.08376499922984221, 0..."
A032903147O439YA3R0D,1.0,"[-0.011696874023991076, -0.014122395600793545,...",A032903147O439YA3R0D,1.0,"[-0.028432001644433446, 0.020244245146814693, ...",0.5,"[-0.02006443783421226, 0.0030609247730105744, ..."
A100CQXJ6D44T9,1.0,"[0.07993089395999163, -0.0542324205675629, 0.1...",A100CQXJ6D44T9,1.0,"[-0.025891581786008942, 0.02268636859820494, 0...",0.5,"[0.027019656086991344, -0.01577302598467898, 0..."
A100MPD67FD3ID,1.0,"[-0.005536118080176104, 0.13938347357185474, -...",A100MPD67FD3ID,1.0,"[-0.02516184721071499, 0.024691265202689668, 0...",0.5,"[-0.015348982645445548, 0.0820373693872722, -0..."
...,...,...,...,...,...,...,...
AZWLJC4YOJDYW,1.0,"[-0.012263824034034199, -0.17065938535497122, ...",AZWLJC4YOJDYW,1.0,"[-0.02876155495175431, 0.015063927703036153, 0...",0.5,"[-0.020512689492894254, -0.07779772882596753, ..."
AZWQOKJLSRFTB,1.0,"[-0.0121745599533533, -0.09485913442567812, -0...",AZWQOKJLSRFTB,1.0,"[-0.025078583103170182, 0.014990007053998316, ...",0.5,"[-0.01862657152826174, -0.0399345636858399, -0..."
AZY3B95680TUC,1.0,"[0.018329393908584162, 0.22391680820337428, -0...",AZY3B95680TUC,1.0,"[-0.025125783173002188, 0.024251767134054195, ...",0.5,"[-0.003398194632209013, 0.12408428766871424, -..."
AZYYFN06QYPC9,1.0,"[-0.04118857697605252, 0.033883898676644986, -...",AZYYFN06QYPC9,1.0,"[-0.031379539776324475, 0.023373452948742064, ...",0.5,"[-0.0362840583761885, 0.028628675812693523, -0..."


In [ ]:
#creation of ub benchmark matix for users
x = cu['userId']
xl = cu['CU_b']
y = du_t['userId']
yl = du_t['du_b']
ub_x = pd.concat([x, xl], axis=1)
ub_y = pd.concat([y, yl], axis=1)
ub_x.rename(columns = {'CU_b':'latent_factors'}, inplace = True)
ub_y.rename(columns = {'du_b':'latent_factors'}, inplace = True)
ub = pd.concat([ub_x, ub_y], axis=0)
ub

,userId,latent_factors
A1725TI23A8AZ5,A1725TI23A8AZ5,"[-0.04458072086426748, -0.04418920146217742, -..."
A17FLA8HQOFVIG,A17FLA8HQOFVIG,"[-0.04562376857442771, 0.03590931947979391, -0..."
A18DX1AT72SY95,A18DX1AT72SY95,"[-0.029064811339074632, -0.06232275819789859, ..."
A18OUQEK7IZ2F2,A18OUQEK7IZ2F2,"[0.0382946130088997, 0.015162036419019635, 0.0..."
A197527ILE0LL8,A197527ILE0LL8,"[-0.07163249468294909, 0.045233471028753136, -..."
...,...,...
AZWLJC4YOJDYW,AZWLJC4YOJDYW,"[-0.020512689492894254, -0.07779772882596753, ..."
AZWQOKJLSRFTB,AZWQOKJLSRFTB,"[-0.01862657152826174, -0.0399345636858399, -0..."
AZY3B95680TUC,AZY3B95680TUC,"[-0.003398194632209013, 0.12408428766871424, -..."
AZYYFN06QYPC9,AZYYFN06QYPC9,"[-0.0362840583761885, 0.028628675812693523, -0..."


In [ ]:
du.to_csv( "/content/drive/MyDrive/Colab Notebooks/dcdcsr_du.csv", index=False)

In [ ]:
ub.to_csv( "/content/drive/MyDrive/Colab Notebooks/dcdcsr_ub.csv", index=False)

## mapping process

In [ ]:
# normalise the latent factor matrix ut and benchmark matrix ub into [-1, +1] by using mapminmax function
from sklearn import preprocessing
print(ut.shape)
scaler = preprocessing.MinMaxScaler(feature_range=(-1, 1))
ut_s = scaler.fit_transform(ut)
vt_s = scaler.fit_transform(vt)
ub_a = []
for i in range(len(ub)):
  ub_a.append(numpy.array(ub['latent_factors'][i]))
ub_s = scaler.fit_transform(ub_a)

(9788, 10)


In [ ]:
print(ub_s.shape)
print(ut_s.shape)
print(vt_s.shape)

(9788, 10)
(9788, 10)
(70, 10)


In [ ]:
import pandas
from keras.models import Sequential
from keras import layers
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [ ]:
X = ut_s
Y = ub_s

In [ ]:
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(10, input_dim=10, kernel_initializer='normal', activation='tanh'))
	model.add(Dense(1, kernel_initializer='normal'))
	# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam')
	return model

In [ ]:
estimator = KerasRegressor(build_fn=baseline_model, epochs=10, batch_size=5, verbose=0)
kfold = KFold(n_splits=10)
results = cross_val_score(estimator, X, Y, cv=kfold)
print("Baseline: %.2f (%.2f) MSE" % (results.mean(), results.std()))

In [ ]:
estimator = KerasRegressor(build_fn=baseline_model, epochs=10, batch_size=32, verbose=0)
model = Sequential()
model.add(layers.Dense(10, input_dim=10, kernel_initializer='normal', activation='tanh'))
model.add(layers.Dense(10, input_dim=10, kernel_initializer='normal', activation='tanh'))
model.add(layers.Dense(10, input_dim=10, kernel_initializer='normal', activation='tanh'))
model.add(layers.Dense(10, kernel_initializer='normal'))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
history = model.fit(X, Y, epochs=10, batch_size=64, validation_split=0.2)
accr = model.evaluate(X, Y)
print('Train set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))
kfold = KFold(n_splits=10)
results = cross_val_score(estimator, X, Y, cv=kfold)
print("Baseline: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Epoch 1/10
245/245 [==============================] - 1s 2ms/step - loss: 0.0626 - accuracy: 0.1826 - val_loss: 0.0622 - val_accuracy: 0.1854
Epoch 2/10
245/245 [==============================] - 0s 2ms/step - loss: 0.0615 - accuracy: 0.1895 - val_loss: 0.0623 - val_accuracy: 0.1854
Epoch 3/10
245/245 [==============================] - 0s 2ms/step - loss: 0.0615 - accuracy: 0.1895 - val_loss: 0.0623 - val_accuracy: 0.1854
Epoch 4/10
245/245 [==============================] - 0s 2ms/step - loss: 0.0615 - accuracy: 0.1895 - val_loss: 0.0622 - val_accuracy: 0.1854
Epoch 5/10
245/245 [==============================] - 0s 2ms/step - loss: 0.0615 - accuracy: 0.1895 - val_loss: 0.0623 - val_accuracy: 0.1854
Epoch 6/10
245/245 [==============================] - 0s 2ms/step - loss: 0.0615 - accuracy: 0.1895 - val_loss: 0.0624 - val_accuracy: 0.1854
Epoch 7/10
245/245 [==============================] - 0s 2ms/step - loss: 0.0615 - accuracy: 0.1895 - val_loss: 0.0623 - val_accuracy: 0.1854
Epoch 

In [ ]:
!pip install keras_tuner
import keras_tuner as kt
import matplotlib.pyplot as plt

In [ ]:
def build_model(hp):
    model = Sequential()
    model.add(layers.Dense(10, input_dim=10, kernel_initializer='normal', activation='tanh'))
    model.add(layers.Dense(10, input_dim=10, kernel_initializer='normal', activation='tanh'))
    model.add(layers.Dense(10, input_dim=10, kernel_initializer='normal', activation='tanh'))
    model.add(layers.Dense(10, kernel_initializer='normal'))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
    return model
tuner = kt.RandomSearch(build_model, objective='val_loss', max_trials=100)    

kfold = KFold(n_splits=10, shuffle=True)
fold_no = 1
acc_per_fold = []
loss_per_fold = []

for train, test in kfold.split(X, Y):
  tuner.search(X, Y, epochs=10, validation_data=(X[0:500], Y[0:500]))

  best_model_ktune = tuner.get_best_models(1)[0]
  best_hyperparameters_ktune = tuner.get_best_hyperparameters(1)[0]
  print(best_model_ktune, best_hyperparameters_ktune)
  scores = best_model_ktune.evaluate(X[0:500], Y[0:500], verbose=0)
  acc_per_fold.append(scores[1])
  loss_per_fold.append(scores[0])
  fold_no = fold_no + 1

accr = best_model_ktune.evaluate(X, Y)
print('Train set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Trial 1 Complete [00h 00m 05s]
val_loss: 0.06662913411855698

Best val_loss So Far: 0.06662913411855698
Total elapsed time: 00h 00m 05s
INFO:tensorflow:Oracle triggered exit
<keras.engine.sequential.Sequential object at 0x7f0e76202f90> <keras_tuner.engine.hyperparameters.HyperParameters object at 0x7f0e778fb310>
INFO:tensorflow:Oracle triggered exit
<keras.engine.sequential.Sequential object at 0x7f0e6ef05fd0> <keras_tuner.engine.hyperparameters.HyperParameters object at 0x7f0e778fb310>
INFO:tensorflow:Oracle triggered exit
<keras.engine.sequential.Sequential object at 0x7f0e779e4450> <keras_tuner.engine.hyperparameters.HyperParameters object at 0x7f0e778fb310>
INFO:tensorflow:Oracle triggered exit
<keras.engine.sequential.Sequential object at 0x7f0e84979b50> <keras_tuner.engine.hyperparameters.HyperParameters object at 0x7f0e778fb310>
INFO:tensorflow:Oracle triggered exit
<keras.engine.sequential.Sequential object at 0x7f0e6ef62210> <keras_tuner.engine.hyperparameters.HyperParameters 

In [ ]:
z = model.predict(X)
z

array([[-0.07023709,  0.14028369, -0.02792341, ...,  0.03928266,
         0.00129442,  0.08423768],
       [-0.06951456,  0.14083333, -0.02784634, ...,  0.03811468,
         0.00086629,  0.08394133],
       [-0.06943044,  0.14009309, -0.02784562, ...,  0.03837188,
         0.00090279,  0.08331285],
       ...,
       [-0.06971559,  0.13952376, -0.02774671, ...,  0.03864551,
         0.00121352,  0.08332592],
       [-0.06915399,  0.14091092, -0.02771281, ...,  0.03748002,
         0.00082667,  0.08373713],
       [-0.06947655,  0.13932194, -0.0277711 , ...,  0.03847215,
         0.00128815,  0.08300473]], dtype=float32)

In [ ]:
path = r'/content/drive/MyDrive/Colab Notebooks/dcdcsr_ub.csv'
with open(path, encoding="utf-8", errors='ignore') as infile:
  ppp = pd.read_csv(infile)

In [ ]:
ppp

,userId,latent_factors
0,A1725TI23A8AZ5,"[-0.04458072086426748, -0.04418920146217742, -..."
1,A17FLA8HQOFVIG,"[-0.04562376857442771, 0.03590931947979391, -0..."
2,A18DX1AT72SY95,"[-0.029064811339074632, -0.06232275819789859, ..."
3,A18OUQEK7IZ2F2,"[0.0382946130088997, 0.015162036419019635, 0.0..."
4,A197527ILE0LL8,"[-0.07163249468294909, 0.045233471028753136, -..."
...,...,...
9783,AZWLJC4YOJDYW,"[-0.020512689492894254, -0.07779772882596753, ..."
9784,AZWQOKJLSRFTB,"[-0.01862657152826174, -0.0399345636858399, -0..."
9785,AZY3B95680TUC,"[-0.003398194632209013, 0.12408428766871424, -..."
9786,AZYYFN06QYPC9,"[-0.0362840583761885, 0.028628675812693523, -0..."


#Tri Factorization

In [ ]:
#Non-probabilistic non-negative matrix tri-factorisation, as presented in
#"Probabilistic Matrix Tri-Factorisation" (Yoo and Choi, 2009).
#We change the notation to match ours: R = FSG.T instead of V = USV.T.
#The updates are then:
#- Uik <- Uik * (sum_j Vjk * Rij / (Ui dot Vj)) / (sum_j Vjk)
#- Vjk <- Vjk * (sum_i Uik * Rij / (Ui dot Vj)) / (sum_i Uik)
#Or more efficiently using matrix operations:
#- Uik <- Uik * (Mi dot [V.k * Ri / (Ui dot V.T)]) / (Mi dot V.k)
#- Vjk <- Vjk * (M.j dot [U.k * R.j / (U dot Vj)]) / (M.j dot U.k)
#And realising that elements in each column in U and V are independent:
#- U.k <- U.k * sum(M * [V.k * (R / (U dot V.T))], axis=1) / sum(M dot V.k, axis=1)
#- V.k <- V.k * sum(M * [U.k * (R / (U dot V.T))], axis=0) / sum(M dot U.k, axis=0)
#We expect the following arguments:
#- R, the matrix
#- M, the mask matrix indicating observed values (1) and unobserved ones (0)
#- K, the number of row latent factors
#- L, the number of column latent factors
    
#Initialisation can be done by running the initialise(init,tauUV) function. We initialise as follows:
#- init_S = 'ones'          -> S[i,k] = 1
#        = 'random'        -> S[i,k] ~ U(0,1)
#         = 'exponential'   -> S[i,k] ~ Exp(expo_prior)
#- init_FG = 'ones'          -> F[i,k] = G[j,k] = 1
#          = 'random'        -> F[i,k] ~ U(0,1), G[j,l] ~ G(0,1), 
#          = 'exponential'   -> F[i,k] ~ Exp(expo_prior), G[j,l] ~ Exp(expo_prior) 
#          = 'kmeans'        -> F = KMeans(R,rows)+0.2, G = KMeans(R,columns)+0.2
#  where expo_prior is an additional parameter (default 1)
#"""

! pip install kmeans
from kmeans.kmeans import KMeans
from distributions.exponential import exponential_draw

import numpy,itertools,math,time

class NMTF:
    def __init__(self,R,M,K,L):
        self.R = numpy.array(R,dtype=float)
        self.M = numpy.array(M,dtype=float)
        self.K = K            
        self.L = L    
        
        self.metrics = ['MSE','R^2','Rp']
                
        assert len(self.R.shape) == 2, "Input matrix R is not a two-dimensional array, " \
            "but instead %s-dimensional." % len(self.R.shape)
        assert self.R.shape == self.M.shape, "Input matrix R is not of the same size as " \
            "the indicator matrix M: %s and %s respectively." % (self.R.shape,self.M.shape)
        
        (self.I,self.J) = self.R.shape
        
        self.check_empty_rows_columns() 
        
        # For computing the I-div it is better if unknown values are 1's, not 0's
        self.R_excl_unknown = numpy.empty((self.I,self.J))
        for i,j in itertools.product(range(0,self.I),range(0,self.J)):
            self.R_excl_unknown[i,j] = self.R[i,j] if self.M[i,j] else 1.
                 
                 
    # Raise an exception if an entire row or column is empty
    def check_empty_rows_columns(self):
        sums_columns = self.M.sum(axis=0)
        sums_rows = self.M.sum(axis=1)
                    
        # Assert none of the rows or columns are entirely unknown values
        for i,c in enumerate(sums_rows):
            assert c != 0, "Fully unobserved row in R, row %s." % i
        for j,c in enumerate(sums_columns):
            assert c != 0, "Fully unobserved column in R, column %s." % j
                 

#    """ Initialise F, S and G """    
    def initialise(self,init_S='random',init_FG='random',expo_prior=1.):
        assert init_S in ['ones','random','exponential'], "Unrecognised init option for S: %s." % init_S
        assert init_FG in ['ones','random','exponential','kmeans'], "Unrecognised init option for F,G: %s." % init_FG
        
        if init_S == 'ones':
            self.S = numpy.ones((self.K,self.L))
        elif init_S == 'random':
            self.S = numpy.random.rand(self.K,self.L)
        elif init_S == 'exponential':
            self.S = numpy.empty((self.K,self.L))
            for k,l in itertools.product(xrange(0,self.K),xrange(0,self.L)):        
                self.S[k,l] = exponential_draw(expo_prior)
        
        if init_FG == 'ones':
            self.F = numpy.ones((self.I,self.K))
            self.G = numpy.ones((self.J,self.L))
        elif init_FG == 'random':
            self.F = numpy.random.rand(self.I,self.K)
            self.G = numpy.random.rand(self.J,self.L)
        elif init_FG == 'exponential':
            self.F = numpy.empty((self.I,self.K))
            self.G = numpy.empty((self.J,self.L))
            for i,k in itertools.product(xrange(0,self.I),xrange(0,self.K)):        
                self.F[i,k] = exponential_draw(expo_prior)
            for j,l in itertools.product(xrange(0,self.J),xrange(0,self.L)):
                self.G[j,l] = exponential_draw(expo_prior)
        elif init_FG == 'kmeans':
            print("Initialising F using KMeans.")
            kmeans_F = KMeans(self.R,self.M,self.K)
            kmeans_F.initialise()
            kmeans_F.cluster()
            self.F = kmeans_F.clustering_results + 0.2            
            
            print("Initialising G using KMeans.")
            kmeans_G = KMeans(self.R.T,self.M.T,self.L)   
            kmeans_G.initialise()
            kmeans_G.cluster()
            self.G = kmeans_G.clustering_results + 0.2
        
        
#    """ Update F, S, G for a number of iterations, printing the performances each iteration. """
    def run(self,iterations):
        assert hasattr(self,'F') and hasattr(self,'S') and hasattr(self,'G'), \
            "F, S and G have not been initialised - please run NMTF.initialise() first."        
        
        self.all_times = [] # to plot performance against time
        self.all_performances = {} # for plotting convergence of metrics
        for metric in self.metrics:
            self.all_performances[metric] = []
            
        time_start = time.time()
        for it in range(1,iterations+1):
            # Doing S first gives more interpretable results (F,G ~= [0,1] rather than [0,20])
            for k,l in itertools.product(xrange(0,self.K),xrange(0,self.L)):
                self.update_S(k,l)
                    
            for k in range(0,self.K):
                self.update_F(k)
                
            for l in range(0,self.L):
                self.update_G(l)
               
            self.give_update(it)
            
            time_iteration = time.time()
            self.all_times.append(time_iteration-time_start)  
        
        
#    """ Method for doing both initialise() and run() """
    def train(self,iterations,init_S='random',init_FG='random',expo_prior=1.):
        self.initialise(init_S=init_S,init_FG=init_FG,expo_prior=expo_prior) 
        self.run(iterations=iterations)
        
                
#    """ Updates for F, G, S. """                
    # Compute the dot product of three matrices
    def triple_dot(self,M1,M2,M3):
        return numpy.dot(M1,numpy.dot(M2,M3))
        
    def update_F(self,k):
        R_pred = self.triple_dot(self.F,self.S,self.G.T)
        SG = numpy.dot(self.S[k],self.G.T)
        numerator = (self.M * self.R / R_pred * SG).sum(axis=1)
        denominator = (self.M * SG).sum(axis=1)
        self.F[:,k] = self.F[:,k] * numerator / denominator
        
    def update_G(self,l):
        R_pred = self.triple_dot(self.F,self.S,self.G.T)
        FS = numpy.dot(self.F,self.S[:,l])
        numerator = ((self.M * self.R / R_pred).T * FS).T.sum(axis=0)
        denominator = (self.M.T * FS).T.sum(axis=0)
        self.G[:,l] = self.G[:,l] * numerator / denominator
        
    def update_S(self,k,l):
        R_pred = self.triple_dot(self.F,self.S,self.G.T)
        F_times_G = self.M * numpy.outer(self.F[:,k], self.G[:,l])   
        numerator = (self.R * F_times_G / R_pred).sum()
        denominator = F_times_G.sum()
        self.S[k,l] = self.S[k,l] * numerator / denominator
           
           
#    ''' Functions for computing MSE, R^2 (coefficient of determination), Rp (Pearson correlation) '''
    def predict(self,M_pred):
        R_pred = self.triple_dot(self.F,self.S,self.G.T)
        MSE = self.compute_MSE(M_pred,self.R,R_pred)
        R2 = self.compute_R2(M_pred,self.R,R_pred)    
        Rp = self.compute_Rp(M_pred,self.R,R_pred)        
        return {'MSE':MSE,'R^2':R2,'Rp':Rp}        
        
    def compute_MSE(self,M,R,R_pred):
        return (M * (R-R_pred)**2).sum() / float(M.sum())
        
    def compute_R2(self,M,R,R_pred):
        mean = (M*R).sum() / float(M.sum())
        SS_total = float((M*(R-mean)**2).sum())
        SS_res = float((M*(R-R_pred)**2).sum())
        return 1. - SS_res / SS_total if SS_total != 0. else numpy.inf
        
    def compute_Rp(self,M,R,R_pred):
        mean_real = (M*R).sum() / float(M.sum())
        mean_pred = (M*R_pred).sum() / float(M.sum())
        covariance = (M*(R-mean_real)*(R_pred-mean_pred)).sum()
        variance_real = (M*(R-mean_real)**2).sum()
        variance_pred = (M*(R_pred-mean_pred)**2).sum()
        return covariance / float(math.sqrt(variance_real)*math.sqrt(variance_pred))   
        
    def compute_I_div(self):    
        R_pred = self.triple_dot(self.F,self.S,self.G.T)
        return (self.M * ( self.R_excl_unknown * numpy.log( self.R_excl_unknown / R_pred ) - self.R_excl_unknown + R_pred ) ).sum()        
        
        
#    """ Give updates and store performances """
    def give_update(self,iteration):    
        perf = self.predict(self.M)
        i_div = self.compute_I_div()
        
        for metric in self.metrics:
            self.all_performances[metric].append(perf[metric])
               
        print("Iteration %s. I-divergence: %s. MSE: %s. R^2: %s. Rp: %s." % (iteration,i_div,perf['MSE'],perf['R^2'],perf['Rp']))

ModuleNotFoundError: ignored

In [ ]:
import kmeans.kmeans

ModuleNotFoundError: ignored

#testing

In [ ]:
print(rs)
print(rt)

userId      A0334855HN6E38CXWXZR  ...  AZYH2ZK8UE1YY
itemId                            ...               
000047715X                     0  ...              0
000105001X                     0  ...              0
000171287X                     0  ...              0
000211495X                     0  ...              0
000216132X                     0  ...              0
...                          ...  ...            ...
2153971                        0  ...              0
2160595                        0  ...              0
2163578                        0  ...              0
230022                         0  ...              0
913154                         0  ...              0

[67 rows x 9702 columns]
userId      A0034986DWR7WEDQN0GV  ...  AZZXJAE2DILET
itemId                            ...               
000503860X                     0  ...              0
061524226X                     0  ...              0
076700941X                     0  ...              0
076780192X          

In [ ]:
arr_s = rs.to_numpy()
arr_t = rt.to_numpy()
arr_s

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
x = np.matmul(arr_t, arr_t.transpose())
x.shape

(290, 290)